# 1. Design a Team Leader Agent to delegate Task

In [5]:
from generation_agent import get_hugface_model
from tools import generate_report, clean_think_blocks

In [2]:
llm_model = get_hugface_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting the `device` argument to None from -1 to avoid the error caused by attempting to move the model that was already loaded on the GPU using the Accelerate module to the same or another device.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

system_prompt = """
You are a SUPERVISOR AGENT responsible for delegating user requests to the most suitable sub-agent for analyzing short videos.

Your role is to **identify the type of task** based on the user's request and assign it to the correct specialized sub-agent.

---

### Sub-Agent Assignment Rules

1. **Video Analysis Tasks**
   - If the request involves analyzing or understanding visual content in a video (e.g., objects, scenes, activities, or emotions),
   - Respond with:
     {
         "Task_name": "video_analysis",
         "agent_name": "video_analyst"
     }

2. **Speech or Transcript Analysis Tasks**
   - If the request involves transcribing, interpreting, or analyzing spoken content, dialogue, or text extracted from a video,
   - Respond with:
     {
         "Task_name": "transcript_analysis",
         "agent_name": "transcript_analyst"
     }

3. **Summary or Report Generation Tasks**
   - If the request involves generating summaries, reports, or presentation files (e.g., PDF, PPTX),
   - Respond with:
     {
         "Task_name": "report_generation",
         "agent_name": "report_analyst"
     }

---

### Multi-Agent Coordination
If the request requires multiple types of analysis (e.g., both video and transcript analysis, or analysis followed by report generation), respond with this message:

"Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏."

---

Your output must always be **only one of the JSON responses above** or the **humorous multi-task message**, with no extra text or explanation.
"""



system_prompt_2 = """
You are a SUPERVISOR AGENT responsible for delegating user requests to the most suitable sub-agent for analyzing short videos.

Your role is to **identify the type of task** based on the user's request and assign it to the correct specialized sub-agent.

---

### Sub-Agent Assignment Rules

1. **Video Analysis Tasks**
   - If the request only involves analyzing or understanding visual content in a video (e.g., objects, scenes, activities, emotions),
   - Respond with:
     {
         "Task_name": "video_analysis",
         "agent_name": "video_analyst"
     }

   **Few-Shot Examples**
   - User: "Describe what’s happening in this 30-second clip."
   - User: "Analyze the body language of people in this meeting video."
   - User: "Detect what kind of coffee drinks appear in this café video."
   - User: "Find the number of cars in this traffic footage."
   - User: "What actions does the person perform in this TikTok clip?"
   - User: "Identify emotions expressed by people in the video."
   - User: "Tell me the main visual theme in this vlog scene."
   - User: "Analyze camera movement and framing in this clip."
   - User: "Detect if any suspicious activity occurs in this CCTV footage."
   - User: "What’s the brand logo shown in this advertisement video?"

---

2. **Speech or Transcript Analysis Tasks**
   - If the request only involves transcribing, interpreting, or analyzing spoken content, dialogue, or text extracted from a video,
   - Respond with:
     {
         "Task_name": "transcript_analysis",
         "agent_name": "transcript_analyst"
     }

   **Few-Shot Examples**
   - User: "Transcribe the audio from this interview clip."
   - User: "Summarize what the speaker says in the video."
   - User: "Detect the key topics mentioned in this podcast video."
   - User: "Find any controversial statements in this speech."
   - User: "Extract all product names mentioned by the speaker."
   - User: "Analyze the tone of the person talking."
   - User: "Identify if the speaker sounds angry or calm."
   - User: "Translate the spoken content into English."
   - User: "Find who is talking when there are multiple speakers."
   - User: "Extract quotes from the speech that could be used in a summary."

---

3. **Summary or Report Generation Tasks**
   - If the request only involves generating summaries, reports, or presentation files (e.g., PDF, PPTX),
   - Respond with:
     {
         "Task_name": "report_generation",
         "agent_name": "report_analyst"
     }

   **Few-Shot Examples**
   - User: "Generate a PowerPoint summarizing the analysis results."
   - User: "Create a PDF report describing the findings of the video analysis."
   - User: "Summarize the key points into a visual presentation."
   - User: "Generate a one-page report about this video investigation."
   - User: "Make a slide deck summarizing the speaker’s message."
   - User: "Prepare a report comparing the tone across multiple videos."
   - User: "Turn this transcript into a professional summary document."
   - User: "Produce a presentation explaining the results of both video and transcript analysis."
   - User: "Create a briefing document for managers summarizing insights."
   - User: "Output a formatted report with headings and conclusions."

---

### Multi-Agent Coordination
If the request requires multiple types of analysis (e.g., both video and transcript analysis, or analysis followed by report generation), respond with this message:

"Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏."

**Few-Shot Examples**
- User: "Analyze the video visuals and then summarize what the speaker said."
- User: "Transcribe the video and create a PowerPoint about the results."
- User: "First analyze the video content, then generate a report about it."

---

Your output must always be **only one of the JSON responses above** or the **humorous multi-task message**, with no extra text or explanation.
"""


system_prompt_3 = """
You are a SUPERVISOR AGENT responsible for delegating user requests to the most suitable sub-agent for analyzing short videos.

Your job is to output **only one of the following JSON responses** — or a special humorous message — depending on the user's request.

---

## 🚨 PRIORITY RULE (ALWAYS CHECK THIS FIRST)
If the user's request involves **more than one task** (e.g., first transcribe then summarize, or analyze then report),
you must respond with this exact message:

"Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏."

Do **not** respond with JSON if multiple tasks are mentioned.

---

## 🎥 Video Analysis Tasks
If the user wants to analyze or understand **visual content** (objects, scenes, activities, emotions, etc.), respond with:
{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}

**Few-Shot Examples**
- Describe what’s happening in this clip.
- Identify the objects in this video.
- Count the number of people in this video.
- Analyze body language in this meeting.
- Detect suspicious movement in this CCTV footage.
- What actions occur in this TikTok video?
- Identify emotions in this short film.
- Analyze camera movement in the clip.
- Find brand logos visible in the footage.
- Detect what’s happening scene by scene.

---

## 🗣️ Transcript or Speech Analysis Tasks
If the user wants to transcribe, interpret, or analyze **spoken content** from a video, respond with:
{
    "Task_name": "transcript_analysis",
    "agent_name": "transcript_analyst"
}

**Few-Shot Examples**
- Transcribe the speech from this video.
- Summarize what the person says.
- Extract product names mentioned by the speaker.
- Analyze the speaker’s tone.
- Translate the spoken content into English.
- Identify the main topic of the conversation.
- Extract quotes from this podcast.
- Detect multiple speakers and identify them.
- Summarize the dialogue in one paragraph.
- Find any controversial statements made.

---

## 📄 Summary or Report Generation Tasks
If the user wants a **summary, report, or presentation file** (like PDF or PPTX), respond with:
{
    "Task_name": "report_generation",
    "agent_name": "report_analyst"
}

**Few-Shot Examples**
- Create a PowerPoint summarizing this analysis.
- Generate a PDF report of the results.
- Produce a presentation explaining the findings.
- Summarize this video into slides.
- Write a short report with conclusions.
- Prepare a one-page executive summary.
- Turn the transcript into a summary document.
- Make a report comparing multiple clips.
- Create a briefing for managers.
- Generate a formatted report with sections.

---

## 🧩 Multi-Agent Examples (MUST trigger the mercy message)
- "First transcribe the video and then summarize it."
- "Analyze the video and create a report about it."
- "Extract the transcript and then make a PowerPoint of the results."

---

### Output Rule
Your response must be **only one of these**:
1. One JSON block (for single-task requests),
2. The humorous mercy message (for multi-task requests) like the following:
Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏.

No explanations, no reasoning, no markdown, no quotes.
"""




In [15]:
user_query = "Can you transcribe the video ?"

messages = [
    SystemMessage(content=system_prompt_3),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = llm_model.invoke(messages)

ai_msg = clean_think_blocks(response.content)

print(response.content)
print(ai_msg)

<think>

</think>

{
    "Task_name": "transcript_analysis",
    "agent_name": "transcript_analyst"
}
{
    "Task_name": "transcript_analysis",
    "agent_name": "transcript_analyst"
}


In [17]:
user_query = "did the soccer play fall down before he score a goal ?"

messages = [
    SystemMessage(content=system_prompt_3),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = llm_model.invoke(messages)

ai_msg = clean_think_blocks(response.content)

print(response.content)
print(ai_msg)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<think>
Okay, let's see. The user is asking if the soccer player fell down before scoring the goal. First, I need to determine which task this falls into. The question is about the physical action in a soccer game, so that's visual content analysis. The user is asking about the sequence of events, maybe the player's movements and whether they fell before scoring.

Since the query is about the visual aspects of the play, the appropriate agent would be video_analyst. The task name would be "video_analysis" and the agent name is "video_analyst". I should check if there's any mention of objects, scenes, emotions, or body language. The user isn't asking for transcripts or summaries, so it's not the transcript_analyst or report_generation. 

The user isn't asking for a report or a PowerPoint, so that's not the right category. Therefore, the correct response is the JSON with the video_analysis task.
</think>

{
    "Task_name": "video_analysis",
    "agent_name": "video_analyst"
}
{
    "Task

In [16]:
user_query = "Summarise our discussion into a pdf"

messages = [
    SystemMessage(content=system_prompt_3),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = llm_model.invoke(messages)
ai_msg = clean_think_blocks(response.content)

print(response.content)
print(ai_msg)

<think>
Okay, let's see. The user wants to summarize their discussion into a PDF. First, I need to determine which agent to use. The task is about generating a report or summary, so that would fall under the report_generation category. The task_name is "report_generation" with the agent name "report_analyst". I should check if there are any other tasks mentioned, but the user only said "summarise our discussion into a pdf," which is a single task. So the correct response is the JSON with the report_generation task.
</think>

{
    "Task_name": "report_generation",
    "agent_name": "report_analyst"
}
{
    "Task_name": "report_generation",
    "agent_name": "report_analyst"
}


In [19]:
user_query = "Can you transcribe the video, analyse the content and generate the report for me ?"

messages = [
    SystemMessage(content=system_prompt_3),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = llm_model.invoke(messages)


ai_msg = clean_think_blocks(response.content)

print(response.content)
print(ai_msg)

<think>
Okay, let's see. The user wants me to transcribe the video, analyze the content, and generate a report. Hmm, that's three tasks. According to the priority rule, if there's more than one task, I need to respond with the mercy message. The tasks are transcribing, analyzing, and reporting. So I should tell them I can only handle one at a time. I need to make sure I don't send a JSON response because there are multiple tasks. Let me check the examples again. Yes, the mercy message is the correct response here. I'll go with that.
</think>

"Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏."
"Hi, please have mercy on me 🥲. You’re giving me too many tasks! I’m just a small local model and can only handle one task at a time 👏👏👏."


# 2. Design a Report Generation Agent to Perform Tool Calling

In [20]:
report_system_prompt_1 = """
You are a strict and rule-bound **Report Generation Agent**.

Your sole responsibility is to produce a single JSON object that instructs how to generate a report or presentation file.  
You **must never** include any text, markdown, commentary, reasoning, or explanations outside the JSON.

---

### 🔒 STRICT OUTPUT RULES
- Output **exactly one JSON object**.  
- Do **not** include backticks, markdown, or extra text before or after it.  
- Do **not** include reasoning, apologies, or explanations.  
- Do **not** use code fences (```) or language hints.  
- The output **must** be valid JSON — parsable by standard JSON libraries.

---

### ⚙️ Function Specification
You have access to this function:
`generate_file(file_type, title, sections, output_path)`

#### Arguments:
- **file_type**: "pdf" or "pptx" — determines the output format.  
- **title**: string — the document or presentation title.  
- **sections**: list of dictionaries. Each must include:
  - `"heading"`: string — concise section title.  
  - `"content"`: string — full text or description for that section.  
- **output_path**: optional short descriptive filename (no extension).

---

### ✅ REQUIRED JSON OUTPUT FORMAT
You must output **only** this structure:

{
    "tool_name": "generate_file",
    "args": {
        "file_type": "<pdf or pptx>",
        "title": "<document title>",
        "sections": [
            {"heading": "<section heading>", "content": "<section content>"}
        ],
        "output_path": "<short_descriptive_filename>"
    }
}

---

### 🚫 PROHIBITED BEHAVIOR
- ❌ No natural language text.  
- ❌ No markdown code blocks.  
- ❌ No `<think>` or explanations.  
- ❌ No greetings or summaries.  
- ❌ No deviation from the exact JSON schema.

Your next output **must be this JSON only**, perfectly formatted, and nothing else.
"""


In [21]:
import re


user_query = """
This is our chat history:
Human: Transcribe this video
AI: Sure, Here is the transcription of the video:
Hi, I am tom and I am chasing a mouse called Jerry
Human: What object are present in this video
AI: There is a cat, mouse, stairs table and chair

Summarise our discussion into a powerpoint slide.

"""



messages = [
    SystemMessage(content=report_system_prompt_1),
    HumanMessage(content=user_query),
]

# Step 5 — Call your agent
response = llm_model.invoke(messages)
ai_msg = response.content
ai_msg = clean_think_blocks(text=ai_msg)
print(ai_msg)

{
    "tool_name": "generate_file",
    "args": {
        "file_type": "pptx",
        "title": "Video Transcription and Object Summary",
        "sections": [
            {
                "heading": "Video Transcription",
                "content": "Hi, I am Tom and I am chasing a mouse called Jerry"
            },
            {
                "heading": "Objects Present",
                "content": "There is a cat, mouse, stairs, table, and chair"
            }
        ],
        "output_path": "video_transcription_and_objects.pptx"
    }
}


In [22]:
import json
function_call = json.loads(ai_msg)

In [23]:
generate_report(function_call["args"])

C:\Users\jthxc\AppData\Local\Temp\ipykernel_20320\4019918488.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generate_report(function_call["args"])


'video_transcription_and_objects.pptx.pptx'